In [1]:
import sys
sys.path.insert(0, r'../quetzal')

from quetzal.io.gtfs_reader import importer

In [1]:
import sys
sys.path.insert(0, r'../../../quetzal')

from quetzal.io.gtfs_reader import importer
from quetzal.io import excel
from quetzal.model import stepmodel, model
from syspy.io.geojson_utils import gdf_to_geojson

from shapely.geometry import Point, MultiPoint, LineString
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
file_name = 'gtfs-Al_Ula-Phase_3_-_2024_RATP_Dev_-_V2' #'gtfs-Al_Ula-Ambition_2035_0.1_gtfs_export_20231129' #'gtfs-Al_Ula-Ambition_2035_0.1_gtfs_export_20231129' #'gtfs-Al_Ula-Phase_3_-_2024_RATP_Dev' #gtfs-Al_Ula-Base_0.1_indicators_no_adjusted_capa_20231211

In [3]:
gtfs = importer.GtfsImporter(path=r'./'+ file_name + '.zip', dist_units='m')
gtfs = gtfs.restrict_to_timerange(['05:00:00', '23:59:59'])
gtfs.describe()

,indicator,value
0,agencies,[Agency created by default]
1,running_services,"[Weekend, Weekday]"
2,timezone,Europe/Paris
3,start_date,20220314
4,end_date,20220318
5,num_routes,19
6,num_trips,45
7,num_stops,202
8,num_shapes,35
9,num_frequencies,194


In [4]:
from datetime import datetime
for date in gtfs.calendar_dates.date:
    d = datetime.strptime(date, '%Y%m%d').strftime('%Y%m%d')
    print(d, type(d))

20220318 <class 'str'>
20220314 <class 'str'>


In [5]:
gtfs.validate()

,type,message,table,rows
1,error,Invalid feed_publisher_name; maybe has extra s...,feed_info,[0]
2,error,Invalid feed_publisher_url; maybe has extra sp...,feed_info,[0]
3,error,Invalid feed_lang; maybe has extra space chara...,feed_info,[0]
8,error,Invalid route_type; maybe has extra space char...,routes,"[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
0,warning,Unrecognized column feed_id,agency,[]
4,warning,Unrecognized column feed_id,routes,[]
5,warning,Unrecognized column buffer_size,routes,[]
6,warning,Unrecognized column speed,routes,[]
7,warning,Unrecognized column station_stop_time,routes,[]


Pb avec le valideur de GTFS en ligne, la table calendar.txt : impossible to read csv file 'calendar.txt': CSV deserialize error: record 1 (line: 2, byte: 88): Invalid value ``, expected 0 or 1. Il faut la modifier en ajoutant Weekday du Samedi au mercredi, jeudi et vendredi étant des Weekend.
Start date = 14 mars 2022 un lundi
End date = 18 mars 2022 un vendredi

In [6]:
new_calendar = pd.DataFrame({"service_id": ["Weekend", "Weekday"], 
                             "monday" :[0,1],
                             "tuesday" :[0,1],
                             "wednesday" :[0,1],
                             "thursday" :[1,0],
                             "friday" :[1,0],
                             "saturday" :[0,1],
                             "sunday" :[0,1],
                             "start_date": [20220314, 20220314],
                             "end_date": [20220318, 20220318]
                             })
new_calendar

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,Weekend,0,0,0,1,1,0,0,20220314,20220318
1,Weekday,1,1,1,0,0,1,1,20220314,20220318


In [7]:
gtfs.calendar = new_calendar

In [8]:
# gtfs.write(file_name + "_corrected.zip")

Idem, on modifie calendar_dates.txt avec 
- en colonne service_id "Weekend" ou "Weekday"
- en colonne date "20220318" ou "20220314"
- en exception_type "1" car le service a été ajouté pour la date spécifié (et non 2 = supprimé pour la date spécifié)

In [9]:
gtfs.calendar_dates

,service_id,date,exception_type
0,Weekday,20220318,1
1,Weekend,20220314,1


_pas nécessaire_ Idem, on modifie calendar_dates.txt avec 
- en colonne service_id "Weekend" ou "Weekday"
- en colonne date "20220318" ou "20220314"
- en exception_type "1" car le service a été ajouté pour la date spécifié (et non 2 = supprimé pour la date spécifié)

In [11]:
gtfs.trips.head()

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,c21f75a6-aecc-11ee-bf1e-02420aca9e06,Weekday,081eac16-aecd-11ee-a929-02420aca9e06,trip for pattern,default,1,default,248527,0,0
1,c21f75a6-aecc-11ee-bf1e-02420aca9e06,Weekday,0e278d48-b09a-11ee-8212-02420aca9e07,trip for pattern,default,1,default,248528,0,0
2,8af525ca-aecd-11ee-833d-02420aca9e07,Weekday,f95943b2-2401-11ef-9bb9-02420ac9af03,trip for pattern,default,1,default,248571,0,0
3,8af525ca-aecd-11ee-833d-02420aca9e07,Weekday,5977c16e-2403-11ef-8611-02420ac9af07,trip for pattern,default,1,default,248572,0,0
4,aa77bcfe-aece-11ee-ad7d-02420aca9e03,Weekday,ea57097e-aece-11ee-8820-02420aca9e04,trip for pattern,default,1,default,248530,0,0


In [12]:
link_route_trips = pd.merge(gtfs.routes, gtfs.trips, on = 'route_id')

In [13]:
gpd_shapes = gtfs.shapes
gpd_shapes = gpd.GeoDataFrame(gpd_shapes, geometry=gpd.points_from_xy(gpd_shapes.shape_pt_lon, gpd_shapes.shape_pt_lat))
gpd_shapes = gpd_shapes.merge(link_route_trips[['route_short_name', 'shape_id']], on = 'shape_id')
gpd_shapes = gpd_shapes.merge(gtfs.routes[['route_short_name','route_color']], on ='route_short_name')
gpd_shapes = gpd_shapes.groupby(['route_color', 'shape_id', 'route_short_name'], as_index = False)['geometry'].apply(lambda x: LineString(x.tolist()))
gpd_shapes = gpd.GeoDataFrame(gpd_shapes, geometry='geometry')
gpd_shapes.rename(columns={"route_short_name":"Transport"}, inplace=True)
gpd_shapes.head()

,route_color,shape_id,Transport,geometry
0,000000,248559,IC1,"LINESTRING (37.90529 26.66508, 37.90529 26.665..."
1,000000,248560,IC1,"LINESTRING (39.29482 25.68193, 39.29482 25.681..."
2,000000,248561,IC2,"LINESTRING (37.90529 26.66508, 37.90529 26.665..."
3,000000,248562,IC2,"LINESTRING (38.29629 26.78348, 38.29629 26.783..."
4,000000,248563,IC3,"LINESTRING (37.90529 26.66508, 37.90529 26.665..."


In [14]:
gpd_shapes['Transport'] = gpd_shapes['Transport'].str.replace("GRT", "DRT")
gpd_shapes.sort_values(by = 'Transport', inplace = True, ascending = True)
gpd_shapes_NODRT = gpd_shapes[(gpd_shapes['Transport'].str.find("DRT")<0) & (gpd_shapes['Transport'].str.find("DRDO")<0)]

gpd_shapes_NODRT.to_file('itineraries_'+ file_name + '_nodrt.geojson', drive = "GeoJSON")

# gpd_shapes_DRT = gpd_shapes[(gpd_shapes['Transport'].str.find("DRT")>=0) | (gpd_shapes['Transport'].str.find("DRDO")>=0)]

# gpd_shapes_DRT.to_file('itineraries_'+ file_name + '_drt.geojson', drive = "GeoJSON")
# gpd_shapes_DRT.to_file('itineraries_'+ file_name + '_drt.geojson', drive = "GeoJSON")

In [15]:
gpd_shapes.crs

In [15]:
# 

# TODO: 
- automatiser unie mise en page des lignes, afficher les arrêts sur la carte, et avoir une couche de buffer automatique de 300m, 400m sur QGIS selon le mode. Calculer les indicateurs de couvertures ligne par ligne.
- trouver le HUBs, si indirect = créer des buffers autour des arrêts de 30 ou 50 mètres, combien de lignes ? vérifier leur accessibilité sur l'outil.
- calculer les KPI comme interdistance moyenne selon les types de ligne (shuttles, bus, brt...)
- sur l'outil, ajuster les vitesse commerciale, les temps d'arrêt, puis la fréquence selon les hp, 

sortir le tableau à partir des gtfs :
Line type	
Purpose	Detailed description	
L#	
Stations number	Peak hour headway	
Length	
Trip time (one way)	
Avg. Commercial speed	
PPHPD	
Estimated OPEX	
Type of road	
Inhab.	
Jobs	
Tour.	
Veh.	
km produced	Connexion with

In [16]:
capa_dict = dict(
    {'F':60,
     'H': 20, #Harrat Cable
     'C': 60,
     'B': 90,
     'X': 60,
     'T': 140,
     'J': 140,
     'L': 300,
     'G': 10,
     'D': 10,
     'I': 60,
     'R':90,
     'V':60,
     'A':10}
)

interval_dict = dict(
    {
        'A1':10,
        'A2':10,
        'A3':10,
        'Tram Phase 1':6,
        'Tram Phase 2':6,
        'Tram Phase 2 Option 1':6,
        'Tram Phase 2 Option 2':6,
        'JTT tram':6,
        'LRT':6,
        'Harrat Cable transport ': 20,
        'B1':5,
        'B2':15,
        'B3':6,
        'B4':15,
        'B5':10,   #attention, avec Sc 1 version 0.1, 15, Sc 1 version 0.2 = 10
        'B6':15,
        'B7':0,
        'B8':0,
        'X1':6,
        'X2':6,
        'X2 w/TPh2':6,
        'X Airport':5,         # ignorer dans l'analyse de couverture
        'C1':5,        # ignorer dans l'analyse de couverture
        'C2':25,
        'C3':10,        # ignorer dans l'analyse de couverture
        'C4':10,        # ignorer dans l'analyse de couverture
        'C5':6,
        'C6':10,
        'C7':0,
        'C8':0,
        'AlUla Central Loop':5,
        'F1':10,
        'F2':10,
        'F3':15,
        'F4':15,
        'F5':15,
        'F6':15,
        'F7':0,
        'F8':0,
        'FX':20,
        'GRT Central 1' : 3,
        'GRT Central 2' : 3,
        'GRT Central 3' : 3,
        'GRT Central 4' : 3,
        'GRT East 5' : 3,
        'GRT East 6' : 3,
        'GRT South 7' : 3,
        'GRT South 8' : 3,
        'GRT South 9' : 3,
        'GRT South 10' : 3,
        'GRT South 11' : 3,
        'GRT South 12' : 3,
        'GRT South 13' : 3,
        'GRT South 14' : 3,
        'GRT North 15 Equestrian village': 3,
        'GRT North 16 Ashar Valley':3,
        'DRT Central 1' : 3,
        'DRT Central 2' : 3,
        'DRT Central 3' : 3,
        'DRT Central 4' : 3,
        'DRT East 5' : 3,
        'DRT East 6' : 3,
        'DRT South 7' : 3,
        'DRT South 8' : 3,
        'DRT South 9' : 3,
        'DRT South 10' : 3,
        'DRT South 11' : 3,
        'DRT South 12' : 3,
        'DRT South 13' : 3,
        'DRT South 14' : 3,
        'DRT North 15 Equestrian village': 3,
        'DRT North 16 Ashar Valley':3,
        'DRDO':3,
        'R1': 20,
        'R2': 20,
        'R3': 20,
        'R4': 20,
        'R5': 20,
        'R6': 20,
        'V1': 20,
        'V2': 20,
        'V3': 20,
        'V4': 20,
        'V5': 20,
        'V6': 20,
        'V7': 20,
        'V8': 20,
        'V9 - Blue Line': 20,
        'V9 - Yellow Line': 20,
        'V9 - Red Line': 20,
        'V10': 20,
        'V11': 20,
        'IC1': 20,
        'IC2': 20,
        'IC3': 20,
        'IC4': 20,
        'IC5': 20
    }
)

In [17]:
def convert_to_mn(hhmmss):
    return int(hhmmss[0:2])*60 + int(hhmmss[3:5]) + int(hhmmss[6:8])/60
def convert_to_hhmmss(mm):
    h = str(int(mm//60)).zfill(2)
    m = str(int(mm%60)).zfill(2)
    s = str(int((mm%60 - int(mm%60))*60)).zfill(2)
    return h +':' + m + ':' + s

In [18]:
final_tab = gpd_shapes.copy()
final_tab = final_tab.set_crs(crs=4326)
final_tab = final_tab.to_crs(epsg=2318)
final_tab['average_length_km'] = final_tab['geometry'].length/1000
final_tab = final_tab.to_crs(crs=4326)
final_tab = final_tab[['Transport', 'average_length_km']]
final_tab = final_tab.groupby(['Transport'], as_index = False).mean()
final_tab['average_length_km'] = final_tab.average_length_km.apply(lambda x : (np.round(x,1)))
final_tab

,Transport,average_length_km
0,A1,2.0
1,A2,1.5
2,A3,1.8
3,B1,6.2
4,B2,11.1
5,B3,10.0
6,B4,14.6
7,B5,7.5
8,C1,24.1
9,C2,28.2


In [19]:
final_tab = gpd_shapes.copy()
final_tab = final_tab.set_crs(crs=4326)
final_tab = final_tab.to_crs(epsg=2318)
final_tab['average_length_km'] = final_tab['geometry'].length/1000
final_tab = final_tab.to_crs(crs=4326)
final_tab = final_tab[['Transport', 'average_length_km']]
final_tab = final_tab.groupby(['Transport'], as_index = False).mean()
final_tab['average_length_km'] = final_tab.average_length_km.apply(lambda x : (np.round(x,1)))
final_tab['Capacity'] = final_tab.Transport.apply(lambda x: capa_dict[x[0]])
final_tab['Interval_mn'] = final_tab.Transport.apply(lambda x: interval_dict[x])
final_tab['PPHPD'] = (60/final_tab['Interval_mn'] * final_tab['Capacity']).apply(lambda x : int(np.round(x,1)))

link_route_trips_stops = link_route_trips.merge(gtfs.stop_times, on = 'trip_id')
link_route_trips_stops = link_route_trips_stops.merge(gpd_shapes, on = 'shape_id')

link_route_trips_stops = link_route_trips_stops[[
    # 'feed_id', 'agency_id', 'route_short_name', 'route_long_name', 'route_desc', 'route_url', 'route_color', 'route_text_color', 
    # 'wheelchair_accessible', 'trip_headsign', 'service_id', 'trip_short_name','bikes_allowed', 'stop_headsign', 
    # 'pickup_type', 'drop_off_type', 'timepoint', 'geometry', 'speed','buffer_size', 'block_id', 'shape_id', 
    'route_id', 'trip_id',
    'route_type', 
    'station_stop_time',
    # 'direction_id', 
    'arrival_time', 'departure_time', 'stop_id',
    'stop_sequence', 
    'shape_dist_traveled', 'Transport'
]]
stop_tab = link_route_trips_stops.groupby(['Transport'], as_index=False)['stop_sequence'].max()
stop_tab.rename(columns={'stop_sequence': 'number_of_stops'}, inplace = True)

link_route_trips_stops_end = link_route_trips_stops.merge(stop_tab, on = 'Transport')
link_route_trips_stops_end = link_route_trips_stops_end[link_route_trips_stops_end.stop_sequence == link_route_trips_stops_end.number_of_stops]
link_route_trips_stops_end['travel_time'] = link_route_trips_stops_end['arrival_time'].apply(convert_to_mn)
link_route_trips_stops_end = link_route_trips_stops_end.groupby(['Transport', 'number_of_stops'], as_index= False)['travel_time'].mean()
final_tab = final_tab.merge(link_route_trips_stops_end, on = 'Transport')
final_tab['Com_speed_kmph'] = (final_tab['average_length_km']/(final_tab['travel_time']/60)).apply(lambda x : int(np.round(x,0)))
final_tab['travel_time'] = final_tab['travel_time'].apply(convert_to_hhmmss)
final_tab['dist_between_stops_m'] = (final_tab['average_length_km']/(final_tab['number_of_stops']-1)*1000).apply(lambda x: int(np.round(x,0)))
# final_tab.sort_values(by = 'dist_between_stops', ascending = False)
final_tab.sort_values(by = 'Transport', inplace = True, ascending = True)
final_tab['Transport'] = final_tab['Transport'].str.replace("GRT", "DRT")
#[link_route_trips_stops.arrival_time.apply(lambda x: int(x[0:2])) < 9]

# CHANGEMENT: F2, C4, C1, X1, F3

In [20]:
final_tab

,Transport,average_length_km,Capacity,Interval_mn,PPHPD,number_of_stops,travel_time,Com_speed_kmph,dist_between_stops_m
0,A1,2.0,10,10,60,2,00:08:13,15,2000
1,A2,1.5,10,10,60,2,00:06:41,13,1500
2,A3,1.8,10,10,60,2,00:07:35,14,1800
3,B1,6.2,90,5,1080,14,00:19:17,19,477
4,B2,11.1,90,15,360,26,00:39:02,17,444
5,B3,10.0,90,6,900,22,00:35:20,17,476
6,B4,14.6,90,15,360,27,00:53:06,16,562
7,B5,7.5,90,10,540,16,00:21:56,21,500
8,C1,24.1,60,5,720,15,00:45:44,32,1721
9,C2,28.2,60,25,144,7,00:27:47,61,4700


In [21]:
# gtfs.map_stops(gtfs.stops.stop_id)
stop_gpd = gpd.GeoDataFrame(gtfs.stops, geometry=gpd.points_from_xy(gtfs.stops.stop_lon, gtfs.stops.stop_lat))
# stop_gpd.to_file('stops_' + file_name + '.geojson', drive = "GeoJSON")

In [22]:
tab = final_tab.copy()
tab['Mode'] = tab['Transport'].apply(lambda x: x[0])
tab = tab.drop(['travel_time', 'Transport'], axis = 1)
tab['travel_time'] = tab['average_length_km']/tab['Com_speed_kmph']*60
tab = tab.groupby(['Mode'], as_index= False).agg(['min', 'max']).sort_values(by = 'Mode', ascending = True)
tab

average_length_km       Capacity      Interval_mn     PPHPD        \
                   min   max      min  max         min max   min   max   
Mode                                                                     
A                  1.5   2.0       10   10          10  10    60    60   
B                  6.2  14.6       90   90           5  15   360  1080   
C                 11.2  28.7       60   60           5  25   144   720   
F                  9.7  12.6       60   60          10  10   360   360   
H                  3.3   3.3       20   20          20  20    60    60   
J                 22.1  22.1      140  140           6   6  1400  1400   
L                 36.8  36.8      300  300           6   6  3000  3000   
X                 28.7  33.3       60   60           5   6   600   720   

     number_of_stops     Com_speed_kmph     dist_between_stops_m         \
                 min max            min max                  min    max   
Mode                                                                      
A                  2   2             13  15                 1500   2000   
B                 14  27             16  21                  444    562   
C                  2  16             32  61                 1018  28700   
F                 12  22             29  30                  600    882   
H                  2   2             30  30                 3300   3300   
J                 17  17             33  33                 1381   1381   
L                 28  28             34  34                 1363   1363   
X                  6  11             30  34                 3330   5740   

     travel_time             
             min        max  
Mode                         
A       6.923077   8.000000  
B      19.578947  54.750000  
C      20.785714  45.187500  
F      19.400000  26.068966  
H       6.600000   6.600000  
J      40.181818  40.181818  
L      64.941176  64.941176  
X      50.647059  66.600000

In [23]:
# tab['average_length_km']/tab['Com_speed_kmph']*60

Transfer stations :

In [24]:
stop_times_df = gtfs.stop_times
trips_df = gtfs.trips
routes_df = gtfs.routes
stops_df = gtfs.stops
stop_and_routes = pd.merge(stop_times_df, trips_df, on = 'trip_id').merge(routes_df, on = 'route_id').merge(stops_df, on = 'stop_id')
stop_and_routes = stop_and_routes[stop_and_routes['route_short_name'].str.find('GRT')<0]
stop_and_routes = stop_and_routes[stop_and_routes['route_short_name'].str.find('DRDO')<0]
stop_and_routes

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,...,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,geometry
0,0d8a250c-d63e-11ee-bccf-02420acb9204_1,00:00:00,00:00:30,3686976,1,NaN,0,NaN,NaN,NaN,...,NaN,26.630431,37.911531,NaN,NaN,0,NaN,NaN,NaN,POINT (37.91153 26.63043)
1,0d8a250c-d63e-11ee-bccf-02420acb9204_2,00:00:00,00:00:30,3686976,1,NaN,0,NaN,NaN,NaN,...,NaN,26.630431,37.911531,NaN,NaN,0,NaN,NaN,NaN,POINT (37.91153 26.63043)
2,2e7bb41a-d63e-11ee-9309-02420acb9203_1,00:06:41,00:07:11,3686976,2,NaN,0,NaN,1546.4315,NaN,...,NaN,26.630431,37.911531,NaN,NaN,0,NaN,NaN,NaN,POINT (37.91153 26.63043)
3,2e7bb41a-d63e-11ee-9309-02420acb9203_2,00:06:41,00:07:11,3686976,2,NaN,0,NaN,1546.4315,NaN,...,NaN,26.630431,37.911531,NaN,NaN,0,NaN,NaN,NaN,POINT (37.91153 26.63043)
4,80a4b01c-d63d-11ee-9309-02420acb9203_1,00:07:56,00:08:26,3686976,2,NaN,0,NaN,1859.0839,NaN,...,NaN,26.630431,37.911531,NaN,NaN,0,NaN,NaN,NaN,POINT (37.91153 26.63043)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043,f9f45960-cf1a-11ee-ab25-02420acb9207_2,00:03:35,00:03:55,bb425c76-32b7-11ee-a9d1-02420acd2a06,4,NaN,0,NaN,1502.5370,NaN,...,NaN,26.872168,37.843270,NaN,NaN,0,NaN,NaN,NaN,POINT (37.84327 26.87217)
1044,f9f45960-cf1a-11ee-ab25-02420acb9207_1,00:04:39,00:04:59,bb42c404-32b7-11ee-a9d1-02420acd2a06,5,NaN,0,NaN,1939.2363,NaN,...,NaN,26.870913,37.847446,NaN,NaN,0,NaN,NaN,NaN,POINT (37.84745 26.87091)
1045,f9f45960-cf1a-11ee-ab25-02420acb9207_2,00:04:39,00:04:59,bb42c404-32b7-11ee-a9d1-02420acd2a06,5,NaN,0,NaN,1939.2363,NaN,...,NaN,26.870913,37.847446,NaN,NaN,0,NaN,NaN,NaN,POINT (37.84745 26.87091)
1046,f9f45960-cf1a-11ee-ab25-02420acb9207_1,00:05:43,00:06:03,bb4329c6-32b7-11ee-a9d1-02420acd2a06,6,NaN,0,NaN,2360.5627,NaN,...,NaN,26.869334,37.851303,NaN,NaN,0,NaN,NaN,NaN,POINT (37.85130 26.86933)


In [25]:
stop_times_df = gtfs.stop_times
trips_df = gtfs.trips
routes_df = gtfs.routes
stops_df = gtfs.stops
stop_and_routes = pd.merge(stop_times_df, trips_df, on = 'trip_id').merge(routes_df, on = 'route_id').merge(stops_df, on = 'stop_id')
stop_and_routes = stop_and_routes[~stop_and_routes['route_short_name'].str.contains('DRT')]
stop_and_routes = stop_and_routes[~stop_and_routes['route_short_name'].str.contains('DRDO')]
stop_and_routes[stop_and_routes['route_short_name'].str.contains('DRT')].route_short_name

# = stop_and_routes[~stop_and_routes['route_short_name'].str.contains('DRDO')]
# stop_and_routes[stop_and_routes['route_short_name'].str.find('GRT')<0].route_short_name.unique()

Series([], Name: route_short_name, dtype: object)

In [26]:
stop_times_df = gtfs.stop_times
trips_df = gtfs.trips
routes_df = gtfs.routes
stops_df = gtfs.stops
stop_and_routes = pd.merge(stop_times_df, trips_df, on = 'trip_id').merge(routes_df, on = 'route_id').merge(stops_df, on = 'stop_id')
stop_and_routes = stop_and_routes[~stop_and_routes['route_short_name'].str.contains('DRT')]
stop_and_routes = stop_and_routes[~stop_and_routes['route_short_name'].str.contains('DRDO')]

stop_and_routes = stop_and_routes[['route_short_name', 'stop_id', 'stop_name', 'stop_lat', 'stop_lon']]
stop_and_routes = stop_and_routes.drop_duplicates()
stop_and_routes

# stop_and_transport = stop_and_routes.groupby('stop_id')['route_short_name'].apply(tuple).reset_index(name='Transport')
# stop_and_transport

,route_short_name,stop_id,stop_name,stop_lat,stop_lon
0,A2,3686976,Old Al Ula,26.630431,37.911531
4,A1,3686976,Old Al Ula,26.630431,37.911531
8,A3,3686976,Old Al Ula,26.630431,37.911531
12,A2,3686535,Old Al Ula - P+R,26.617768,37.917028
16,B1,3686535,Old Al Ula - P+R,26.617768,37.917028
...,...,...,...,...,...
1038,C4,bb4185b2-32b7-11ee-a9d1-02420acd2a06,stop 1691147407,26.876014,37.835010
1040,C4,bb41f330-32b7-11ee-a9d1-02420acd2a06,stop 1691147409,26.873342,37.839095
1042,C4,bb425c76-32b7-11ee-a9d1-02420acd2a06,stop 1691147410,26.872168,37.843270
1044,C4,bb42c404-32b7-11ee-a9d1-02420acd2a06,stop 1691147413,26.870913,37.847446


In [27]:
stop_times_df = gtfs.stop_times
trips_df = gtfs.trips
routes_df = gtfs.routes
stops_df = gtfs.stops
stop_and_routes = pd.merge(stop_times_df, trips_df, on = 'trip_id').merge(routes_df, on = 'route_id').merge(stops_df, on = 'stop_id')
stop_and_routes = stop_and_routes[~stop_and_routes['route_short_name'].str.contains('DRT')]
stop_and_routes = stop_and_routes[~stop_and_routes['route_short_name'].str.contains('DRDO')]

stop_and_routes = stop_and_routes[['route_short_name', 'stop_id', 'stop_name', 'stop_lat', 'stop_lon']]
stop_and_routes = stop_and_routes.drop_duplicates()
stop_and_transport = stop_and_routes.groupby('stop_id')['route_short_name'].apply(tuple).reset_index(name='Transport')
stop_and_routes = stop_and_routes.groupby(['stop_id', 'stop_name'])['stop_lat', 'stop_lon'].first().reset_index()

stop_and_transport = stop_and_transport.merge(stop_and_routes, on = ['stop_id'])
stop_and_transport = gpd.GeoDataFrame(stop_and_transport, geometry=gpd.points_from_xy(stop_and_transport.stop_lon, stop_and_transport.stop_lat))
# stop_and_transport = stop_and_transport[stop_and_transport['Transport'].str.find('GRT')<0]
stop_and_transport['Transport'] = stop_and_transport.Transport.astype('string')
stop_and_transport['Transport'] = stop_and_transport['Transport'].str.replace(",)", "", regex=False)
stop_and_transport['Transport'] = stop_and_transport['Transport'].str.replace(")", "", regex=False)
stop_and_transport['Transport'] = stop_and_transport['Transport'].str.replace("'", "", regex=False)
stop_and_transport['Transport'] = stop_and_transport['Transport'].str.replace("(", "", regex=False)
stop_and_transport
# stop_and_transport[stop_and_transport['Transport'].str.find(',')>0]
# stop_and_transport.to_file('stops_and_transport_' + file_name + '.geojson', drive = "GeoJSON")
stop_and_transport['Connexions'] = stop_and_transport.Transport.apply(lambda x : x.count(','))
stop_and_transport['PT lines'] = stop_and_transport['Connexions']+1
stop_and_transport = stop_and_transport.sort_values(by='Connexions', ascending=False)
stop_and_transport = stop_and_transport.drop_duplicates(subset='geometry', keep='first')

# stop_and_transport.to_file('Ambition_2035_without_DRT_stops.shp')
# stop_and_transport.to_file('Draft_Ambition_2035_stops.shp')
stop_and_transport[stop_and_transport.Connexions > 0]

,stop_id,Transport,stop_name,stop_lat,stop_lon,geometry,Connexions,PT lines
154,3686535,"A2, B1, X Airport, Harrat Cable transport , LR...",Old Al Ula - P+R,26.617768,37.917028,POINT (37.91703 26.61777),6,7
53,3678376,"X Airport, JTT tram, LRT, B3, B5",Al Ula Hijaz,26.602201,37.941126,POINT (37.94113 26.60220),4,5
97,3678563,"B1, X Airport, JTT tram, B3, X1",Al Ula Oasis,26.617977,37.929600,POINT (37.92960 26.61798),4,5
42,3678361,"JTT tram, C3, F1, F2, C4",Hegra Museum,26.757204,37.931838,POINT (37.93184 26.75720),4,5
98,3678565,"C3, F1, X1, C4",North P+R,26.771105,37.895810,POINT (37.89581 26.77110),3,4
40,3678357,"C3, F1, F2, C4",stop 1691477188,26.754374,37.917771,POINT (37.91777 26.75437),3,4
82,3678505,"C3, F1, F2, C4",stop 1691477664,26.753406,37.913019,POINT (37.91302 26.75341),3,4
80,3678503,"C3, F1, F2, C4",stop 1691477686,26.755281,37.922007,POINT (37.92201 26.75528),3,4
79,3678502,"C3, F1, F2, C4",stop 1691477694,26.756216,37.927181,POINT (37.92718 26.75622),3,4
150,3685526,"C1, JTT tram, A1, X1",Dadan Gateway,26.646674,37.907099,POINT (37.90710 26.64667),3,4


In [28]:
stop_and_transport['Connexions'] = stop_and_transport.Transport.apply(lambda x : x.count(','))
stop_and_transport

,stop_id,Transport,stop_name,stop_lat,stop_lon,geometry,Connexions,PT lines
154,3686535,"A2, B1, X Airport, Harrat Cable transport , LR...",Old Al Ula - P+R,26.617768,37.917028,POINT (37.91703 26.61777),6,7
53,3678376,"X Airport, JTT tram, LRT, B3, B5",Al Ula Hijaz,26.602201,37.941126,POINT (37.94113 26.60220),4,5
97,3678563,"B1, X Airport, JTT tram, B3, X1",Al Ula Oasis,26.617977,37.929600,POINT (37.92960 26.61798),4,5
42,3678361,"JTT tram, C3, F1, F2, C4",Hegra Museum,26.757204,37.931838,POINT (37.93184 26.75720),4,5
98,3678565,"C3, F1, X1, C4",North P+R,26.771105,37.895810,POINT (37.89581 26.77110),3,4
...,...,...,...,...,...,...,...,...
76,3678499,C1,stop 1691476740,26.699530,38.036381,POINT (38.03638 26.69953),0,1
77,3678500,C1,stop 1691476717,26.682046,38.031265,POINT (38.03126 26.68205),0,1
78,3678501,C1,stop 1691476694,26.678987,38.017228,POINT (38.01723 26.67899),0,1
81,3678504,F2,stop 1691477655,26.749617,37.911461,POINT (37.91146 26.74962),0,1


In [29]:
# transfer_stops = stop_and_transport['Transport'] #.to_excel('transfer_stops_' + file_name + '.xlsx')
stop_times_df = gtfs.stop_times
trips_df = gtfs.trips
routes_df = gtfs.routes
stops_df = gtfs.stops
stop_and_routes = pd.merge(stop_times_df, trips_df, on = 'trip_id').merge(routes_df, on = 'route_id').merge(stops_df, on = 'stop_id')
stop_and_routes = stop_and_routes[['route_short_name', 'stop_id', 'stop_name', 'stop_lat', 'stop_lon']]
stop_and_routes = stop_and_routes.drop_duplicates().sort_values(by = 'route_short_name',ascending = True)
stop_and_routes
# transport_names = stop_and_routes.route_short_name.unique().sort_values(ascending = False)
# table = pd.pivot_table(stop_and_routes, values='D', index=['A', 'B'], columns=['C'], aggfunc="sum")   # https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html
# table = pd.pivot_table(stop_and_routes, values='route_short_name', index=['stop_id'], columns=['C'], aggfunc="sum")

,route_short_name,stop_id,stop_name,stop_lat,stop_lon
4,A1,3686976,Old Al Ula,26.630431,37.911531
196,A1,3685526,Dadan Gateway,26.646674,37.907099
0,A2,3686976,Old Al Ula,26.630431,37.911531
12,A2,3686535,Old Al Ula - P+R,26.617768,37.917028
292,A3,3685480,Old Town,26.630926,37.923348
...,...,...,...,...,...
36,X1,3686535,Old Al Ula - P+R,26.617768,37.917028
944,X1,3678379,stop 1694526495,26.733396,37.885262
662,X1,3678374,stop 1690872383,26.834533,37.962106
952,X1,2eddb4be-35bf-11ee-9f45-02420acd2a03,أم طليحة,26.719360,37.889789


Convert to kml

In [30]:
# stop_and_routes['color'] = gtfs.routes[gtfs.routes.route_short_name == stop_and_routes['route_short_name']]['route_color']
routes = gtfs.routes
stops__ = stop_and_routes.merge(routes[['route_short_name','route_color']], on ='route_short_name')

In [31]:
gpd_shapes = gtfs.shapes
gpd_shapes = gpd.GeoDataFrame(gpd_shapes, geometry=gpd.points_from_xy(gpd_shapes.shape_pt_lon, gpd_shapes.shape_pt_lat))
gpd_shapes = gpd_shapes.merge(link_route_trips[['route_short_name', 'shape_id']], on = 'shape_id')
gpd_shapes = gpd_shapes.merge(gtfs.routes[['route_short_name','route_color']], on ='route_short_name')
gpd_shapes = gpd_shapes.groupby(['route_color', 'shape_id', 'route_short_name'], as_index = False)['geometry'].apply(lambda x: LineString(x.tolist()))
gpd_shapes = gpd.GeoDataFrame(gpd_shapes, geometry='geometry')
gpd_shapes.rename(columns={"route_short_name":"Transport"}, inplace=True)
gpd_shapes['Transport'] = gpd_shapes['Transport'].str.replace("GRT", "DRT")
gpd_shapes.sort_values(by = 'Transport', inplace = True, ascending = True)
# gpd_shapes = 
gpd_shapes = gpd_shapes[(gpd_shapes['Transport'].str.find("DRT")<0) & (gpd_shapes['Transport'].str.find("DRDO")<0)]
gpd_shapes = gpd_shapes.reset_index(drop= True)
gpd_shapes

,route_color,shape_id,Transport,geometry
0,7a4944,246657,A1,"LINESTRING (37.91152 26.63042, 37.91152 26.630..."
1,7a4944,246656,A1,"LINESTRING (37.90670 26.64668, 37.90670 26.646..."
2,8d37cc,246659,A2,"LINESTRING (37.91703 26.61777, 37.91703 26.617..."
3,8d37cc,246658,A2,"LINESTRING (37.91153 26.63043, 37.91153 26.630..."
4,07c0bf,246660,A3,"LINESTRING (37.91153 26.63043, 37.91153 26.630..."
5,07c0bf,246661,A3,"LINESTRING (37.92335 26.63093, 37.92335 26.630..."
6,e23ba4,246545,B1,"LINESTRING (37.93290 26.61384, 37.93290 26.613..."
7,e23ba4,246544,B1,"LINESTRING (37.93286 26.61384, 37.93286 26.613..."
8,90301d,246551,B2,"LINESTRING (37.98190 26.52182, 37.98190 26.521..."
9,90301d,246550,B2,"LINESTRING (37.93329 26.55683, 37.93329 26.556..."


In [32]:
stop_and_transport

,stop_id,Transport,stop_name,stop_lat,stop_lon,geometry,Connexions,PT lines
154,3686535,"A2, B1, X Airport, Harrat Cable transport , LR...",Old Al Ula - P+R,26.617768,37.917028,POINT (37.91703 26.61777),6,7
53,3678376,"X Airport, JTT tram, LRT, B3, B5",Al Ula Hijaz,26.602201,37.941126,POINT (37.94113 26.60220),4,5
97,3678563,"B1, X Airport, JTT tram, B3, X1",Al Ula Oasis,26.617977,37.929600,POINT (37.92960 26.61798),4,5
42,3678361,"JTT tram, C3, F1, F2, C4",Hegra Museum,26.757204,37.931838,POINT (37.93184 26.75720),4,5
98,3678565,"C3, F1, X1, C4",North P+R,26.771105,37.895810,POINT (37.89581 26.77110),3,4
...,...,...,...,...,...,...,...,...
76,3678499,C1,stop 1691476740,26.699530,38.036381,POINT (38.03638 26.69953),0,1
77,3678500,C1,stop 1691476717,26.682046,38.031265,POINT (38.03126 26.68205),0,1
78,3678501,C1,stop 1691476694,26.678987,38.017228,POINT (38.01723 26.67899),0,1
81,3678504,F2,stop 1691477655,26.749617,37.911461,POINT (37.91146 26.74962),0,1


In [37]:
import simplekml

kml = simplekml.Kml()
for index, row in stop_and_transport.iterrows():
    # color = row['route_color']
    name = row['stop_name']
    geometry = row['geometry']
    description = 'PT lines: ' + row['Transport']
    pnt = kml.newpoint(name=name, coords=list(geometry.coords), description=description)
    pnt.style.width= row['PT lines']

kml.save('stops_' +file_name + '_new.kml')

In [55]:
import simplekml

kml = simplekml.Kml()
for index, row in gpd_shapes.iterrows():
    color = row['route_color']
    name = row['Transport']
    geometry = row['geometry']

    linestring = kml.newlinestring(name=name, coords=list(geometry.coords), description='')

    linestring.style.linestyle.color = simplekml.Color.hex(color)
    linestring.style.linestyle.width= 5

# kml.save('itineraries_' +file_name + '_new.kml')

In [56]:
# #read kml
# itineraries = gpd_shapes.copy()
# itineraries["colour"] = itineraries["route_color"]
# itineraries["name"] = itineraries["Transport"]
# itineraries.to_file('Draft_Ambition_2035_bus_lines.shp')
# itineraries

In [57]:
# bus = gpd.read_file(r'../GIS Data/bus_roads.geojson')
# bus['index'] = bus.index
# bus.to_file(r'../GIS Data/bus_roads_indexed.geojson', drive = "GeoJSON")

In [58]:
# roads = gpd.read_file(r'../GIS Data/00 mail Fatima 2023 07 07/other_roads.geojson')
# bus['index'] = bus.index
# bus.to_file(r'../GIS Data/bus_roads_indexed.geojson', drive = "GeoJSON")
# roads

# Lire l'excel et le transformer en tableau avec :
0) GENERAL INFORMATION
1) Name
2) Type
3) Length
4) Number of stops
5) Main stops
6) PERFORMANCE
7) Travel time
8) Average commercial speed
9) ROLLING STOCK
10) vehicle size / Capacity 
11) Number of vehicles
12) Average annual mileage per vehicle
13) PERFORMANCE - COVERAGE
14) Covered population (inhabitants)
15) Covered jobs
16) Covered touristic trips
17) CONNEXIONS
18) connected to lines:
19) HUB - Lines listing